In [ ]:
%pip install huggingface_hub

In [ ]:
# 下载模型
from huggingface_hub import snapshot_download

model_id = "Qwen/Qwen1.5-0.5B" 
snapshot_download(
    repo_id=model_id, 
    local_dir="Qwen-0.5b",
    local_dir_use_symlinks=False,
    revision="main",
    use_auth_token="<YOUR_ACCESS_TOKEN>")

使用llama.cpp库进行转换

In [ ]:
# 1. 克隆仓库  终端执行以下命令，与下载的模型在同一目录下
!git clone https://github.com/ggerganov/llama.cpp.git

In [ ]:
# 2.安装依赖并测试
%cd llama.cpp
%pip install -r requirements.txt
!pip install huggingface_hub
%python convert.py -h

In [ ]:
# 3.转换模型格式脚本
!python convert_hf_to_gguf.py ../Qwen-0.5b --outfile Qwen_instruct_0.5b.gguf --outtype f16

In [ ]:
# 4.模型量化
!ollama create -q Q4_K_M mymodel3 -f ./Modelfile

In [ ]:
# 上传至huggingface repo
from huggingface_hub import HfApi
import os

api = HfApi()
HF_ACCESS_TOKEN = "<YOUR_ACCESS_TOKEN>"
#TODO 这里需要设置你的model_id
model_id = "little1d/QWEN-0.5b"

api.create_repo(
    model_id,
    exist_ok=True,
    repo_type="model", # 上傳格式為模型
    use_auth_token=HF_ACCESS_TOKEN,
)
# upload the model to the hub
# upload model name includes the Bailong-instruct-7B in same folder
for file in os.listdir():
    if file.endswith(".gguf"):
        model_name = file.lower()
        api.upload_file(
            repo_id=model_id,
            path_in_repo=model_name,
            path_or_fileobj=f"{os.getcwd()}/{file}",
            repo_type="model", # 上傳格式為模型
            use_auth_token='<YOUR_ACCESS_TOKEN>')

